In [13]:
## Install all the required packages and run the required libraries
##-----------------------------------------------
import pandas as pd
import numpy as np
import skimpy

In [14]:
## List of Functions
##-----------------------------------------------
#read a CSV file
#readCSV(String file):The readCSV function reads the CSV file into workplace
#Arguments: file is the CSV file in the select work directory.
def readCSV(filename):
    csv_data = pd.read_csv(filename)
    return csv_data

In [15]:
#write to CSV file
#writeCSV(table DataSet, String file):The writeCSV function writes the dataframe to a CSV file.
#Arguments: DataSet is the name of dataframe to save, file is the name of the CSV output file.
#Returns a CSV file
def writeCSV(dataset,filename):
     dataset.to_csv(filename)

In [16]:
#select dataset columns for analysis
#selectColumns(table DataSet, string columnName, …): This function selects/keeps the list columns needed for analysis from the dataset. Only the
#list of selected columns/attributes are included in the dataset. 
#Arguments: DataSet is the name of the dataframe, columnName is the name of the column to keep in the dataset. Many can be listed, separated by commas.
#Returns a dataset including only the list of columns/attributes that are selected
def selectColumns(dataset,selectCol):
    return dataset[selectCol]


In [17]:
#delete columns from the dataset
def deleteColumns(dataset,deleteCol):
    return dataset.drop(columns=deleteCol)

In [18]:
#clean column headers
#cleanHeaders(table DataSet)
#This function cleans the headers of the columns from spaces and other special characters.
#It only keeps lower case letters, numbers, and underscores (_). The spaces are replaced by ‘_’ and the special characters are removed. 
#Returns a dataframe with clean header names
def cleanHeaders(dataset):
    return clean_columns(dataset)

In [19]:
#filter rows
#The filter function keeps records/rows based on the conditions specified. 
#Only the rows where the condition is TRUE are kept in the DataSet. 
#The filter function supports multiple functions, for example: ==, >, <, >=, <=, &, | , ! . 
def filterRows(dataset,conditions):
    return dataset.query(conditions)

In [20]:
##remove rows with low frequency
#Column represents the column you want to filter
# freq the threshold value that is used to filter out rows whose count is less than freq.
def removeEventsLowFrequency(dataset,Column,freq):
    return dataset[dataset.groupby(Column)[Column].transform('count').ge(freq)]

In [21]:
#delete traces with number of events less than a specific number (num)
def deleteTracesLengthLessThan(dataset,Column,freq):
    return dataset[dataset.groupby(Column)[Column].transform('count').ge(freq)]

In [ ]:
#delete traces that do not start with a specific start event
def deleteTruncatedTracesStart(dataset,Column,startStr):
    return data.loc[data[Column].str.startswith(startStr)]

In [ ]:
#delete traces that do not start with a specific end event
def deleteTruncatedTracesStart(dataset,Column,startStr):
    return data.loc[data[Column].str.endswith(startStr)]

In [24]:
#delete traces with total duration less than t
def deleteTracesWithTimeLess():
    return 0

In [32]:
#concatenate two columns

# data["test"] = data["event"].astype(str) + data["theme"].astype(str)

def concatenateColumns(dataset,newCol,*cols):
    dataset[newCol]=""
    for col in cols:
        dataset[newCol]=dataset[newCol]+" "+dataset[col].astype(str)
    return  dataset

In [ ]:
def eventIsRepeated():
    return 0

In [ ]:
#keep first event
